In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from itertools import product
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which sis why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('pattern_eda.ipynb', 'mutate' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit, vectorize, float64, uint
# from tslearn.preprocessing import TimeSeriesScalerMeanVariance
# from tslearn.piecewise import PiecewiseAggregateApproximation
# from tslearn.piecewise import SymbolicAggregateApproximation, OneD_SymbolicAggregateApproximation
from scipy.stats import zscore
from sortedcontainers import SortedList, SortedSet 

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 200)
pd.set_option('display.max_columns', 50)

from common_util import RAW_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, DT_BIZ_DAILY_FREQ, df_rows_in_year, get_custom_biz_freq, get_custom_biz_freq_df, query_df, search_df, chained_filter, benchmark
from common_util import MUTATE_DIR, load_df, load_json, outer_join, left_join, count_nn_df, count_nz_df, count_nn_nz_df, pairwise, cust_count, list_get_dict, get_time_mask
from data.data_api import DataAPI
from raw.common import default_row_masksfile
from recon.viz import *
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/mutate/pattern_eda.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


## Load Data

### RAW

In [2]:
VOL_DIR = RAW_DIR +'vol/'
PRICE_DIR = RAW_DIR +'price/'

SPX = load_df(PRICE_DIR +'SPX.parquet')
RUT = load_df(PRICE_DIR +'RUT.parquet')
NDX = load_df(PRICE_DIR +'NDX.parquet')
DJI = load_df(PRICE_DIR +'DJI.parquet')
VIX = load_df(VOL_DIR +'VIX.parquet')
RVX = load_df(VOL_DIR +'RVX.parquet')
VXN = load_df(VOL_DIR +'VXN.parquet')
DVX = load_df(VOL_DIR +'VXD.parquet')

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
pba = {
    'sp_500': SPX,
    'russell_2000': RUT,
    'dow_jones': DJI,
    'nasdaq_100': NDX,    
}
vol = {
    'sp_500': VIX,
    'russell_2000': RVX,
    'dow_jones': DVX,
    'nasdaq_100': VXN,    
}

In [4]:
pba['sp_500']['pba_gmtOffset']

id
1996-01-02 14:00   -6
1996-01-02 15:00   -6
1996-01-02 16:00   -6
1996-01-02 17:00   -6
1996-01-02 18:00   -6
                   ..
2018-01-08 17:00   -6
2018-01-08 18:00   -6
2018-01-08 19:00   -6
2018-01-08 20:00   -6
2018-01-08 21:00   -6
Name: pba_gmtOffset, Length: 44804, dtype: int64

In [5]:
pba['russell_2000']['pba_gmtOffset']

id
1996-01-02 14:00   -6
1996-01-02 15:00   -6
1996-01-02 16:00   -6
1996-01-02 17:00   -6
1996-01-02 18:00   -6
                   ..
2018-01-08 17:00   -5
2018-01-08 18:00   -5
2018-01-08 19:00   -5
2018-01-08 20:00   -5
2018-01-08 21:00   -5
Name: pba_gmtOffset, Length: 46054, dtype: int64

In [6]:
pba['dow_jones']['pba_gmtOffset']

id
1996-01-02 14:00   -5
1996-01-02 15:00   -5
1996-01-02 16:00   -5
1996-01-02 17:00   -5
1996-01-02 18:00   -5
                   ..
2018-01-08 17:00   -5
2018-01-08 18:00   -5
2018-01-08 19:00   -5
2018-01-08 20:00   -5
2018-01-08 21:00   -5
Name: pba_gmtOffset, Length: 45095, dtype: int64

In [7]:
pba['nasdaq_100']['pba_gmtOffset']

id
1996-01-02 14:00   -5
1996-01-02 15:00   -5
1996-01-02 16:00   -5
1996-01-02 17:00   -5
1996-01-02 18:00   -5
                   ..
2018-01-08 18:00   -5
2018-01-08 19:00   -5
2018-01-08 20:00   -5
2018-01-08 21:00   -5
2018-01-08 22:00   -5
Name: pba_gmtOffset, Length: 48000, dtype: int64

In [8]:
def get_local_dti(dti, dest_tz='US/Eastern'):
    conv_dti = dti.tz_convert(dest_tz)
    return conv_dti

In [9]:
ass = 'nasdaq_100'
ex = df_rows_in_year(root_dfs[[ass, 'root', 'root_split_ohlca', 'join', 'pba']]).dropna(how='all')
orig = ex.index
local = get_local_dti(ex.index)
display(orig)
display(local)

NameError: name 'root_dfs' is not defined

### ROOT

In [10]:
root_rcs, root_dfs = DataAPI.axe_load(['root', 'root_split_ohlca'])
hrm_rcs, hrm_dfs = DataAPI.axe_load(['hrm', 'hrm'], lazy=False)

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


## VIEW

In [13]:
day = '2009-01-02'
dayl = ('{} 00:00'.format(day), '{} 23:00'.format(day))
ass = 'nasdaq_100'
raw_pba = pba[ass]
raw_vol = vol[ass]

In [15]:
raw_pba.loc[dayl[0]: dayl[1]].dropna()

,pba_gmtOffset,pba_open,pba_high,pba_low,pba_close,pba_volume,pba_avgPrice,pba_numTrades
id,,,,,,,,
2009-01-02 14:00,-5,1212.74,1216.37,1209.11,1212.33,21176743.0,1213.0478,238.0
2009-01-02 15:00,-5,1212.28,1235.19,1211.26,1234.67,52785444.0,1223.1257,480.0
2009-01-02 16:00,-5,1234.32,1238.97,1231.04,1237.50,78312080.0,1235.3515,479.0
2009-01-02 17:00,-5,1237.50,1246.66,1236.43,1244.09,99656952.0,1241.2140,479.0
2009-01-02 18:00,-5,1244.19,1252.06,1243.47,1249.79,120485908.0,1247.8010,480.0
2009-01-02 19:00,-5,1249.89,1257.75,1247.29,1255.47,149263332.0,1252.7755,478.0
2009-01-02 20:00,-5,1255.58,1266.44,1255.58,1261.18,213983391.0,1259.6859,480.0
2009-01-02 21:00,-5,1263.70,1263.70,1263.70,1263.70,248546028.0,1263.7000,478.0


In [16]:
root_dfs[[ass, 'root', 'root_split_ohlca', 'join', 'pba']].loc[dayl[0]: dayl[1]].dropna()

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2009-01-02 14:00:00+00:00,1212.74,1216.37,1209.11,1212.33,1213.0478
2009-01-02 15:00:00+00:00,1212.28,1235.19,1211.26,1234.67,1223.1257
2009-01-02 16:00:00+00:00,1234.32,1238.97,1231.04,1237.50,1235.3515
2009-01-02 17:00:00+00:00,1237.50,1246.66,1236.43,1244.09,1241.2140
2009-01-02 18:00:00+00:00,1244.19,1252.06,1243.47,1249.79,1247.8010
2009-01-02 19:00:00+00:00,1249.89,1257.75,1247.29,1255.47,1252.7755
2009-01-02 20:00:00+00:00,1255.58,1266.44,1255.58,1261.18,1259.6859


In [27]:
hrm_dfs[[ass, 'hrm', 'hrm', 'pba', 'pba']].loc[dayl[0]: dayl[1]].dropna()

,times
id,
2009-01-02 14:00:00+00:00,2009-01-02 09:00:00+00:00
2009-01-02 15:00:00+00:00,2009-01-02 10:00:00+00:00
2009-01-02 16:00:00+00:00,2009-01-02 11:00:00+00:00
2009-01-02 17:00:00+00:00,2009-01-02 12:00:00+00:00
2009-01-02 18:00:00+00:00,2009-01-02 13:00:00+00:00
2009-01-02 19:00:00+00:00,2009-01-02 14:00:00+00:00
2009-01-02 20:00:00+00:00,2009-01-02 15:00:00+00:00
2009-01-02 21:00:00+00:00,2009-01-02 16:00:00+00:00
